In [1]:
import pandas as pd
pd.options.display.float_format = "{:,.2f}".format

import sys
sys.path.append('../lib/')
from financelib import FinLoad
from financelib import FinCalc
from financelib import FinFetch
from financelib import FinPlot
from financelib import FinInvestmentsGet

from pathlib import Path

import requests # for investments yahoo finance data

import time # sleep fetch

In [2]:
YEAR = 2025
data_path_o = Path("../../tmp/data")

In [3]:
init_holdings = FinLoad.load_init_holdings(data_path_o, YEAR)


In [4]:
df_year_cashflow = FinLoad.load_cashflow(data_path_o, YEAR)

[Errno 2] No such file or directory: '../../tmp/data/2025/cashflow/2025-04_cashflow.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/cashflow/2025-05_cashflow.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/cashflow/2025-06_cashflow.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/cashflow/2025-07_cashflow.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/cashflow/2025-08_cashflow.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/cashflow/2025-09_cashflow.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/cashflow/2025-10_cashflow.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/cashflow/2025-11_cashflow.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/cashflow/2025-12_cashflow.csv'


In [5]:
df_year_investments = FinLoad.load_investments(data_path_o, YEAR)

[Errno 2] No such file or directory: '../../tmp/data/2025/investments/2025-04_investments.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/investments/2025-05_investments.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/investments/2025-06_investments.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/investments/2025-07_investments.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/investments/2025-08_investments.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/investments/2025-09_investments.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/investments/2025-10_investments.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/investments/2025-11_investments.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/investments/2025-12_investments.csv'


In [6]:
df_m_cashflow = FinCalc.calc_monthly_cashflow(df_year_cashflow, init_holdings, YEAR)

In [7]:
df_m_cashflow

,incomes,liabilities,savings,saving_rate,liquidity
2024-12-31,-,-,"14,711.19",-,"14,711.19"
2025-01-31,"2,231.00",-626.02,"1,604.98",0.63,"16,316.17"
2025-02-28,"1,682.25",-616.10,"1,066.15",0.63,"17,382.32"


In [8]:
fig_cashflow = FinPlot.plot_cashflow(df_m_cashflow)
fig_cashflow.show()

### Detail Expenses January 2025

In [9]:
df_expenses = FinCalc.calc_expenses(df_year_cashflow[ df_year_cashflow.index.month == 2 ])
df_expenses["Qty"] = df_expenses.Qty.abs() # sunburst does not understand negative values
fig = FinPlot.plot_expenses_donut(df_expenses)
fig.show()

### Investments 2025

In [10]:
init_holdings = FinLoad.load_init_holdings(data_path_o, YEAR)
df_year_investments = FinLoad.load_investments(data_path_o, YEAR)

[Errno 2] No such file or directory: '../../tmp/data/2025/investments/2025-04_investments.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/investments/2025-05_investments.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/investments/2025-06_investments.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/investments/2025-07_investments.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/investments/2025-08_investments.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/investments/2025-09_investments.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/investments/2025-10_investments.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/investments/2025-11_investments.csv'
[Errno 2] No such file or directory: '../../tmp/data/2025/investments/2025-12_investments.csv'


In [11]:
df_init_investments = FinInvestmentsGet.get_init_holdings_to_df(init_holdings, YEAR)
df_year_investments = pd.concat([df_init_investments, df_year_investments])


In [12]:
df_init_investments

,Type,Symbol,Qty,Category,Subcategory,Description
Date,,,,,,
2024-12-31,Cryptocurrencies,SOL,4.86,Init,Holdings,From previous year
2024-12-31,Cryptocurrencies,ETH,0.01,Init,Holdings,From previous year
2024-12-31,Cryptocurrencies,USDT,108.00,Init,Holdings,From previous year
2024-12-31,ETFs,IE00BK5BQT80,1.00,Init,Holdings,From previous year
2024-12-31,ETFs,JE00B1VS3770,1.00,Init,Holdings,From previous year


In [13]:
df_year_investments

,Type,Symbol,Qty,Category,Subcategory,Description
Date,,,,,,
2024-12-31,Cryptocurrencies,SOL,4.86,Init,Holdings,From previous year
2024-12-31,Cryptocurrencies,ETH,0.01,Init,Holdings,From previous year
2024-12-31,Cryptocurrencies,USDT,108.00,Init,Holdings,From previous year
2024-12-31,ETFs,IE00BK5BQT80,1.00,Init,Holdings,From previous year
2024-12-31,ETFs,JE00B1VS3770,1.00,Init,Holdings,From previous year
2025-01-02,ETFs,IE00BK5BQT80,0.00,State,Holdings,Vanguard FTSE All-World
2025-01-02,ETFs,JE00B1VS3770,0.00,State,Holdings,WisdomTree Gold
2025-01-02,ETFs,IE00BK5BQT80,0.00,State,Holdings,Vanguard FTSE All-World
2025-01-02,ETFs,JE00B1VS3770,0.00,State,Holdings,WisdomTree Gold


In [14]:
holdings_monthlyized = FinInvestmentsGet.get_holdings_monthlyized(df_year_investments, YEAR)


In [15]:

assets_monthlyized = FinInvestmentsGet.get_assets_monthlyized(holdings_monthlyized, data_path_o, YEAR)
assets = FinInvestmentsGet.get_assets_global(assets_monthlyized, holdings_monthlyized)

SOL
../../tmp/data/2025/investments/exchange/SOL-EUR.csv already exists. Data Loaded from local.
ETH
../../tmp/data/2025/investments/exchange/ETH-EUR.csv already exists. Data Loaded from local.
USDT
../../tmp/data/2025/investments/exchange/USDT-EUR.csv already exists. Data Loaded from local.
IE00BK5BQT80
../../tmp/data/2025/investments/exchange/IE00BK5BQT80-EUR.csv already exists. Data Loaded from local.
JE00B1VS3770
../../tmp/data/2025/investments/exchange/JE00B1VS3770-EUR.csv already exists. Data Loaded from local.


In [16]:
holdings_monthlyized['Cryptocurrencies']['SOL']

,Qty,CumQty
2024-12-31,4.86,4.86
2025-01-31,0.00,4.86
2025-02-28,0.00,4.86


In [17]:
assets['Cryptocurrencies']['SOL']

,Open,Close,Returns,Qty,CumQty,Holdings
2024-12-31,230.20,183.58,NaN,4.86,4.86,892.26
2025-01-31,183.60,229.90,0.20,0.00,4.86,"1,117.39"
2025-02-28,229.89,132.40,-0.74,0.00,4.86,643.51


In [18]:
df_year_holdings = FinInvestmentsGet.get_total_holdings(assets)

In [19]:
df_year_holdings

,SOL,ETH,USDT,IE00BK5BQT80,JE00B1VS3770,Total
2024-12-31,892.26,38.43,103.68,133.26,234.53,"1,402.16"
2025-01-31,"1,117.39",37.22,103.68,275.28,252.42,"1,785.99"
2025-02-28,643.51,26.43,103.68,273.22,253.92,"1,300.76"


### NW 2024

In [20]:
df_m_cashflow.T

,2024-12-31,2025-01-31,2025-02-28
incomes,-,"2,231.00","1,682.25"
liabilities,-,-626.02,-616.10
savings,"14,711.19","1,604.98","1,066.15"
saving_rate,-,0.63,0.63
liquidity,"14,711.19","16,316.17","17,382.32"


In [21]:
nw = pd.concat([df_m_cashflow['liquidity'], df_year_holdings['Total']], axis=1, keys=['liquidity', 'investments'])
nw['networth'] = nw.liquidity + nw.investments
nw["nwch"] = (nw.networth - nw.networth.shift(1) )
nw["ch%"] = (nw.networth - nw.networth.shift(1) )/ nw.networth

In [22]:
nw.T

,2024-12-31,2025-01-31,2025-02-28
liquidity,"14,711.19","16,316.17","17,382.32"
investments,"1,402.16","1,785.99","1,300.76"
networth,"16,113.35","18,102.16","18,683.08"
nwch,NaN,"1,988.81",580.92
ch%,NaN,0.11,0.03


### FIRE Metrics on end 2024

In [18]:
current_yearly_expediture = df_m_cashflow.liabilities.sum()
current_nw = nw.nw.iloc[-1]

AttributeError: 'DataFrame' object has no attribute 'nw'

In [48]:
desired_wd = 0.030
fu_number = abs(current_yearly_expediture) / desired_wd
print(fu_number)

256367.3


In [49]:
progress_fi = current_nw / fu_number
print(progress_fi)

0.06179126753841071


In [50]:
current_yearly_saved = nw.nwch.sum()

In [51]:
current_yearly_monthly_savings = current_yearly_saved/12

In [52]:
months_left_to_fire = (fu_number - current_nw)/current_yearly_monthly_savings
print(f"Months left to FIRE: {int(months_left_to_fire)}")

Months left to FIRE: 188
